# 📤 Entregar Trabajo

**Instrucciones:**
1. Completa todos los ejercicios en este notebook
2. Ejecuta todas las celdas
3. Ejecuta las 2 celdas de abajo para entregar

**NO requiere cuenta ni login** - solo ingresa tu nombre y listo.

In [ ]:
import json

# Check if we're in Pyodide (browser environment)
try:
    import pyodide
    print("✅ Pyodide detectado - Estamos en el navegador")
except ImportError:
    print("⚠️ Este notebook solo funciona en JupyterLite")
    print("   Visita: https://juakongdf.github.io/chaos-lab/")
    raise

SERVER_URL = "https://chaos-sub.vercel.app/submit"

def submit_current_notebook():
    """Envía el notebook actual al servidor usando JavaScript"""
    from pyodide.code import run_js
    
    # Get student name from input
    student_name_code = """
    document.getElementById('studentName').value.trim()
    """
    student_name = run_js(student_name_code)
    
    if not student_name:
        run_js("document.getElementById('result').innerHTML = '<div style=\"padding:10px;background:#fee;border-radius:4px;color:#d00\">❌ Por favor ingresa tu nombre</div>'")
        return
    
    run_js("document.getElementById('result').innerHTML = '<div style=\"padding:10px;background:#e7f3ff;border-radius:4px;color:#0066cc\">⏳ Enviando tu trabajo...</div>'")
    
    # Read the notebook file
    try:
        with open("99_Entregar_Trabajo.ipynb", 'r') as f:
            notebook_content = json.load(f)
    except Exception as e:
        error_msg = str(e).replace("'", "\\'").replace('"', '\\"')
        run_js(f"document.getElementById('result').innerHTML = '<div style=\"padding:10px;background:#fee;border-radius:4px;color:#d00\">❌ Error leyendo notebook: {error_msg}</div>'")
        return
    
    # Prepare payload as JSON string
    payload = {
        "student_name": student_name,
        "notebook": notebook_content
    }
    
    payload_json = json.dumps(payload).replace("'", "\\'")
    
    # Send via JavaScript fetch - properly escaped
    js_code = f"""
    (async function() {{
        try {{
            const payload = {payload_json};
            
            const response = await fetch('{SERVER_URL}', {{
                method: 'POST',
                headers: {{
                    'Content-Type': 'application/json',
                }},
                body: JSON.stringify(payload)
            }});
            
            const data = await response.json();
            
            if (response.ok) {{
                document.getElementById('result').innerHTML = 
                    '<div style="padding:15px;background:#d4edda;border-radius:4px;color:#155724;border-left:4px solid #28a745">' +
                    '<strong>✅ ¡Trabajo entregado exitosamente!</strong><br>' +
                    '<small>Archivo: ' + data.filename + '</small></div>';
            }} else {{
                document.getElementById('result').innerHTML = 
                    '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">❌ Error: ' + (data.error || 'Error desconocido') + '</div>';
            }}
        }} catch (error) {{
            console.error('Error completo:', error);
            document.getElementById('result').innerHTML = 
                '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">❌ Error de conexión: ' + error.message + '</div>';
        }}
    }})();
    """
    
    run_js(js_code)
    print("📤 Solicitud enviada al servidor...")

print("✅ Sistema configurado")
print("👇 Ejecuta la celda de abajo para mostrar el formulario")

In [ ]:
from IPython.display import display, HTML
from pyodide.ffi import create_proxy
import js

# Create a JavaScript-callable proxy of the Python function
submit_proxy = create_proxy(submit_current_notebook)

# Register it directly on the window object
js.window.submitNotebook = submit_proxy

form = """
<div style="padding:20px; background:#f0f0f0; border-radius:8px; max-width:500px; margin:20px auto;">
    <h3 style="margin-top:0">📝 Entregar Trabajo</h3>
    <input id="studentName" placeholder="Tu nombre completo" style="width:100%; padding:10px; font-size:16px; margin:10px 0; border:2px solid #ddd; border-radius:4px;"/>
    <button onclick="window.submitNotebook()" style="width:100%; padding:12px; background:#4CAF50; color:white; border:none; border-radius:4px; font-size:16px; cursor:pointer; font-weight:bold;">🚀 Entregar</button>
    <div id="result" style="margin-top:15px;"></div>
</div>
"""
display(HTML(form))
print("✅ Formulario mostrado - Ingresa tu nombre y presiona 'Entregar'")